<ul class="breadcrumb">
  <li><a href="1.2.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="3.2.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font><br> 
  <font size="5" face="Georgia">(with Maternal Effects)</font> 

</div>

In [1]:
using DataFrames,CSV,JWAS,JWAS.Datasets

INFO: Recompiling stale cache file /Users/haocheng/.julia/lib/v0.6/JWAS.ji for module JWAS.


<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multivariate Linear Additive Genetic Model</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [2]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [3]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### pedigree

In [4]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [5]:
#data=readtable(phenofile)
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

In [6]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [7]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [8]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [9]:
model    = build_model(model_equations,R);

In [10]:
set_covariate(model,"age");

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [11]:
ped = get_pedigree(pedfile);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:01

Finished!



calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


In [12]:
set_random(model,"Animal", ped,G)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [13]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [14]:
data[:BW]

4-element Array{Float64,1}:
 100.0
  50.0
 150.0
  40.0

In [15]:
recode(data[:BW],missing=>0.0)

4-element Array{Float64,1}:
 100.0
  50.0
 150.0
  40.0

In [16]:
recode(data[[:BW,:CW]],missing=>0.0)

LoadError: [91mMethodError: no method matching recode(::DataFrames.DataFrame, ::Pair{Missings.Missing,Float64})[0m
Closest candidates are:
  recode([91m::AbstractArray[39m, ::Pair...) at /Users/haocheng/.julia/v0.6/CategoricalArrays/src/recode.jl:328
  recode([91m::CategoricalArrays.CategoricalArray{S,N,R,V,C,U} where U where C where V[39m, ::Any, [91m::Pair...[39m) where {S, N, R} at /Users/haocheng/.julia/v0.6/CategoricalArrays/src/recode.jl:352
  recode([91m::AbstractArray[39m, ::Any, [91m::Pair...[39m) at /Users/haocheng/.julia/v0.6/CategoricalArrays/src/recode.jl:331[39m

In [17]:
out = runMCMC(model,data,chain_length=1000,printout_frequency=500,output_samples_frequency=10);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   1000
starting_value                                false
printout_frequency                              500
output_samples_frequency                         10
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





LoadError: [91mUndefVarError: isna not defined[39m

In [18]:
keys(out)

LoadError: [91mUndefVarError: out not defined[39m

In [19]:
out["MCMC samples for residual covariance matrix"]

LoadError: [91mUndefVarError: out not defined[39m

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. The Multivariate Linear Additive Genetic Model with Maternal Effects</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [20]:
phenofile = Datasets.dataset("testMT","maternal.txt")
data=readtable(phenofile)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #readtable#233(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at /Users/haocheng/.julia/v0.6/DataFrames/src/deprecated.jl:1050
 [3] readtable(::String) at /Users/haocheng/.julia/v0.6/DataFrames/src/deprecated.jl:1050
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] include_string(::Module, ::String, ::String) at /Users/haocheng/.julia/v0.6/Compat/src/Compat.jl:174
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/haocheng/.julia/v0.6/IJulia/src/execute_request.jl:158
 [7] (::Compat.#inner#17{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/haocheng/.julia/v0.6/Compat/src/Compat.jl:488
 [8] eventloop(::ZMQ.Socket) at /Users/haocheng/.julia/v0

,Animal,BW,CW,age,sex,dam
1,S1,100.0,10.0,8,M,0
2,D1,50.0,12.9,7,F,0
3,O1,150.0,13.0,3,M,D1
4,O3,40.0,5.0,4,F,D1


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [21]:
model_equations = "BW = intercept + age + sex + Animal+ dam;
                   CW = intercept + age + sex + Animal";

In [22]:
model = build_model(model_equations,R);
set_covariate(model,"age");

In [23]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
set_random(model,"Animal dam", ped,G0)

LoadError: [91mdam is not found in model equations.[39m

In [24]:
out=runMCMC(model,data,chain_length=1000,printout_frequency=500);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   1000
starting_value                                false
printout_frequency                              500
output_samples_frequency                          0
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





LoadError: [91mUndefVarError: isna not defined[39m

In [25]:
keys(out)

LoadError: [91mUndefVarError: out not defined[39m